In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
from sklearn.linear_model import LinearRegression, SGDRegressor

In [15]:
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

files_id = {
    'housing_data':"1kX_jcLeBpBGvTo8FXDeU2MK-aw1a0voU", #iteration7 data
    'test_housing_data':"1CMsAWhWKWBjI6DDEHtcYmRVZRazfE9bo", #test data for housing
    'ids_com':"10gwiL49calkj-xbx-3rQEK4H2zoJcU11" #ID for the commiting of the project

}

housing_data = pd.read_csv(gd_path(files_id['housing_data']), sep=",")
test_housing_data = pd.read_csv(gd_path(files_id['test_housing_data']), sep=",")
ids_com = pd.read_csv(gd_path(files_id['ids_com']), sep=",")
df = housing_data

# Pre-Processing Pipe

## Ordinary columns

This part was done by hands. Many Bothans died to bring us this information.

In [28]:
from sklearn.model_selection import train_test_split

# X and y creation
#y = df.pop("SalePrice")

# Feature Engineering
X = df

# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [17]:
# Separating numerical features from your training and testing datasets
X_num_train = X_train.select_dtypes(include="number").copy()
X_num_test = X_test.select_dtypes(include="number").copy()


# Define ordinal and one-hot encoding columns
ordinal_cols = X_train.columns.get_indexer(['LandContour', 'LandSlope',
                     'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                     'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                     'HeatingQC', 'KitchenQual', 'Functional',
                     'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
                     'PoolQC'])

onehot_cols = X_train.columns.get_indexer(['MSZoning', 'Condition1', 'Heating', 'Street', 'CentralAir', 
                                         'Foundation', 'Alley', 'LotShape', 'Utilities', 'LotConfig', 
                                         'Neighborhood', 'Condition2', 'BldgType', 'HouseStyle', 
                                         'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
                                         'Electrical', 'GarageType', 'PavedDrive', 'Fence', 'MiscFeature',
                                         'SaleType', 'SaleCondition'])

# Manually establish the order of categories for ordinal features, including "N_A"
ordinal_categories = [['Lvl', 'Bnk', 'HLS', 'Low', 'N_A'],
                      ['Gtl', 'Mod', 'Sev', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Gd', 'Av', 'Mn', 'No', 'N_A'],
                      ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A'],
                      ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Fin', 'RFn', 'Unf', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],
                      ['Ex', 'Gd', 'TA', 'Fa', 'N_A']]

# Create a categorical encoder
categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=ordinal_categories), ordinal_cols),
        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), onehot_cols)
    ]
)

# Create pipelines for numerical and categorical features
numeric_pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    SelectKBest(score_func=f_regression, k=20),
    MinMaxScaler(),
    VarianceThreshold(threshold=0.02)
)

categoric_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N_A"),
    categorical_encoder
)

# Create a preprocessor that applies the appropriate pipeline to each feature type
preprocessor = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num_train.columns),
        ("cat_pipe", categoric_pipe, X_train.columns)
    ]
)



# Models

## Regressions

### SGDRegressor

In [ ]:
sgd_pipeline = make_pipeline(preprocessor,
                              SGDRegressor())

param_grid = {
    'columntransformer__num_pipe__simpleimputer__strategy': ['median'],
    'columntransformer__num_pipe__selectkbest__k': 12,
    'columntransformer__num_pipe__variancethreshold__threshold': [0.005]
}

search = GridSearchCV(sgd_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"SGD" : search.best_score_}
print("Best parameters: ", search.best_params_)

scores

In [30]:
from sklearn.linear_model import LinearRegression, SGDRegressor

sgd_pipeline = make_pipeline(preprocessor,
                            SGDRegressor())

sgd_pipeline.fit(X_train, y_train)

sgd_predictions = sgd_pipeline.predict(X_test)

SGD_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = sgd_predictions)

print("MAPE:", round(SGD_mape,3))

SGR_r2 = r2_score(y_true = y_test,
                    y_pred = sgd_predictions)

print("R-squared:", round(SGR_r2, 3))

MAPE: 0.12
R-squared: 0.849


In [43]:
# Create the preprocessing pipeline
sgd_pipeline = make_pipeline(preprocessor, SGDRegressor())

# Define the hyperparameter grid
param_grid = {
    'columntransformer__num_pipe__simpleimputer__strategy': ['median'],
    'columntransformer__num_pipe__selectkbest__k': [12],
    'columntransformer__num_pipe__variancethreshold__threshold': [0.005]
}

# Perform grid search for hyperparameter tuning
search = GridSearchCV(sgd_pipeline, param_grid, cv=5, verbose=1)
search.fit(X_train, y_train)

# Get the best score and best parameters
scores = {"SGD": search.best_score_}
#print("Best parameters: ", search.best_params_)

# Fit the SGDRegressor pipeline on the training data
sgd_pipeline.fit(X_train, y_train)

# Make predictions on the test data
sgd_predictions = sgd_pipeline.predict(X_test)

# Calculate and print MAPE
SGD_mape = mean_absolute_percentage_error(y_true=y_test, y_pred=sgd_predictions)
print("MAPE:", round(SGD_mape, 3))

# Calculate and print R-squared
SGR_r2 = r2_score(y_true=y_test, y_pred=sgd_predictions)
print("R-squared:", round(SGR_r2, 3))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
MAPE: 0.123
R-squared: 0.843


#### With KBest

### LinearRegression

In [ ]:
full_pipeline = make_pipeline(preprocessor,
                              LinearRegression())

param_grid = {
    'columntransformer__num_pipe__simpleimputer__strategy': ['mean'],
    'columntransformer__num_pipe__selectkbest__k': 4,
    'columntransformer__num_pipe__variancethreshold__threshold': [0.01],
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      verbose=1)

search.fit(X_train, y_train)

scores = {"LR" : search.best_score_}
print("Best parameters: ", search.best_params_)

scores

In [40]:
# Scale the data! 

lr_pipeline = make_pipeline(preprocessor,
                            LinearRegression())

lr_pipeline.fit(X_train, y_train)

lr_predictions = lr_pipeline.predict(X_test)

lr_mape = mean_absolute_percentage_error(y_true = y_test,
                    y_pred = lr_predictions)

print("MAPE:", round(lr_mape,3))

lr_r2 = r2_score(y_true = y_test,
                    y_pred = lr_predictions)

print("R-squared:", round(lr_r2,3))

MAPE: 0.122
R-squared: 0.863


In [47]:
# Create the preprocessing pipeline
lr_pipeline = make_pipeline(preprocessor, LinearRegression())

# Define the hyperparameter grid
param_grid = {
    'columntransformer__num_pipe__simpleimputer__strategy': ['mean'],
    'columntransformer__num_pipe__selectkbest__k': [4],
    'columntransformer__num_pipe__variancethreshold__threshold': [0.01],
}

# Perform grid search for hyperparameter tuning
search = GridSearchCV(sgd_pipeline, param_grid, cv=5, verbose=1)
search.fit(X_train, y_train)

# Get the best score and best parameters
scores = {"SGD": search.best_score_}
#print("Best parameters: ", search.best_params_)

# Fit the SGDRegressor pipeline on the training data
lr_pipeline.fit(X_train, y_train)

# Make predictions on the test data
lr_predictions = lr_pipeline.predict(X_test)

# Calculate and print MAPE
LR_mape = mean_absolute_percentage_error(y_true=y_test, y_pred=sgd_predictions)
print("MAPE:", round(SGD_mape, 3))

# Calculate and print R-squared
LR_r2 = r2_score(y_true=y_test, y_pred=sgd_predictions)
print("R-squared:", round(SGR_r2, 3))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
MAPE: 0.123
R-squared: 0.843


# Iteration 0

In [520]:
y_test_pred_tree = search.predict(X_test)
y_test_pred_knn = search2.predict(X_test)
y_test_pred_rf = search3.predict(X_test)
y_test_pred_SGDR = sgd_pipeline.predict(X_test)
y_test_pred_lr = lr_pipeline.predict(X_test)


baseline_tree_r2 = r2_score(y_test, y_test_pred_tree)
baseline_knn_r2 = r2_score(y_test, y_test_pred_knn)
baseline_rf = r2_score(y_test, y_test_pred_rf)
baseline_SGDR = r2_score(y_test, y_test_pred_SGDR)
baseline_lr = r2_score(y_test, y_test_pred_lr)


performances = pd.DataFrame({'decision_tree': round(baseline_tree_r2,3),
                             'knn': round(baseline_knn_r2,3), 
                             'RF': round(baseline_rf,3),
                             'SGDR': round(baseline_SGDR, 2),
                            'LR': round(baseline_lr,3)},
                            index=['baseline'])

performances

,decision_tree,knn,RF,SGDR,LR
baseline,0.322,0.259,0.673,0.8,0.838


With out Threshold and Skaler

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.372         | -0.229 | 0.697         | -3.505539e+22  | 0.884               |

With Threshold and Skaler

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.454         | 0.108 | 0.595         | 0.84  | 0.866                |


With Threshold and Skaler and GridSearh

| Model       | Decision Tree | KNN   | Random Forest | SGDR  | Logistic Regression |
|-------------|---------------|-------|---------------|-------|----------------------|
| Baseline    | 0.31           | 0.64 | 0.7           | 0.87  |   -7.169538e+19          |

In [521]:
print('Readoy')

Readoy


# Downloand

In [48]:
X_sumbmition = test_housing_data
ids_com['SalePrice'] = lr_pipeline.predict(X_sumbmition) # only cange the piplene! 
ids_com.to_csv(r'submission_lr_grid.csv', index=False) #only cahnge ist